In [27]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import math

import time
import numpy as np
from speech2affective_gestures.net.utils.graph import Graph
from speech2affective_gestures.net.utils.tgcn import STGraphConv, STGraphConvTranspose
import shutil

import librosa
from librosa.feature import mfcc
import numpy as np
from scipy.interpolate import interp1d
from sklearn.preprocessing import normalize

import random

from Model_Components.m2ad_pose_repletion_net import PoseSeq2SeqGenerator, AffDiscriminator

encoder_args = {
    'hidden_size':256,
    'n_layers':5,
    'dropout_prob':0.4
}

decoder_args = {
    'hidden_size':256,
    'n_layers':5,
    'dropout_prob':0.4
}
device = 'cuda'
s2ag_generator = PoseSeq2SeqGenerator(encoder_args, decoder_args, 440, 37, 440, 12, 2, 100).to(device=device)
s2ag_discriminator = AffDiscriminator(2,100, 16).to(device=device)



In [28]:
s2ag_generator.load_state_dict(torch.load('./BN_RN_TFModif/M2AD_Gen11_ep6_10FPS'))
s2ag_discriminator.load_state_dict(torch.load('./BN_RN_TFModif/M2AD_Disc11_ep6_10FPS'))

<All keys matched successfully>

In [29]:
for name, layer in s2ag_generator.named_modules():
    print(name)


encoder
encoder.audio_encoder
encoder.audio_encoder.conv1
encoder.audio_encoder.batch_norm1
encoder.audio_encoder.conv2
encoder.audio_encoder.batch_norm2
encoder.audio_encoder.conv3
encoder.audio_encoder.batch_norm3
encoder.audio_encoder.conv4
encoder.audio_encoder.batch_norm4
encoder.audio_encoder.linear1
encoder.audio_encoder.activation
encoder.chroma_encoder
encoder.chroma_encoder.conv1
encoder.chroma_encoder.batch_norm1
encoder.chroma_encoder.conv2
encoder.chroma_encoder.batch_norm2
encoder.chroma_encoder.conv3
encoder.chroma_encoder.batch_norm3
encoder.chroma_encoder.conv4
encoder.chroma_encoder.batch_norm4
encoder.chroma_encoder.linear1
encoder.chroma_encoder.activation
encoder.aff_encoder
encoder.aff_encoder.st_gcn1
encoder.aff_encoder.st_gcn1.gcn
encoder.aff_encoder.st_gcn1.gcn.conv
encoder.aff_encoder.st_gcn1.tcn
encoder.aff_encoder.st_gcn1.tcn.0
encoder.aff_encoder.st_gcn1.tcn.1
encoder.aff_encoder.st_gcn1.tcn.2
encoder.aff_encoder.st_gcn1.tcn.3
encoder.aff_encoder.st_gcn1.t

In [30]:
s2ag_generator.decoder.out[3].weight

Parameter containing:
tensor([[-0.0311,  0.0611,  0.0524,  ...,  0.0047,  0.0135,  0.0958],
        [ 0.0413,  0.0309,  0.0668,  ...,  0.0485,  0.0689,  0.0137],
        [-0.0136, -0.0277,  0.0326,  ...,  0.0262,  0.0183, -0.0461],
        ...,
        [-0.0502, -0.0755, -0.0661,  ...,  0.0404,  0.0035, -0.0703],
        [-0.0411,  0.0570, -0.0661,  ..., -0.0294, -0.0042,  0.0594],
        [-0.0395,  0.1009, -0.0478,  ..., -0.0556,  0.0433, -0.0152]],
       device='cuda:0', requires_grad=True)